# Yolo (You Only Look Once)

In [33]:
import os
import re
import cv2
import uuid
import logging
import subprocess
from pathlib import Path
from ultralytics import YOLO
from roboflow import Roboflow, Project
from typing import Any, Literal, Pattern
from roboflow.core.dataset import Dataset
from roboflow.core.version import Version
from supervision.detection.core import Detections
from torch.cuda import is_available, empty_cache

ModuleNotFoundError: No module named 'ultralytics'

In [4]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class DirectoryUtil:
    """Utility class for handling directories."""

    @staticmethod
    def directory_exists(path: str) -> bool:
        """
        Checks if the specified directory exists.
        
        Args:
            path (str): The directory path.
        
        Returns:
            bool: True if the directory exists, False otherwise.
        """
        return os.path.isdir(s=path)

    @staticmethod
    def create_directory(path: str) -> None:
        """
        Creates a directory at the specified path.
        If the directory already exists, no exception is raised.
        
        Args:
            path (str): The directory path.
        """
        try:
            os.makedirs(name=path, exist_ok=True)
            logging.info(msg=f"Directory created: {path}")
        except Exception as e:
            logging.info(msg=f"Error creating directory {path}: {e}")

    @staticmethod
    def ensure_directory(path: str) -> str:
        """
        Ensures that the directory exists. If it does not, the directory is created.
        
        Args:
            path (str): The directory path.

        Return:
            str: Directory path.
        """
        if not DirectoryUtil.directory_exists(path):
            DirectoryUtil.create_directory(path)
            return path
        else:
            logging.info(msg=f"Directory already exists: {path}")
            return path

    @staticmethod
    def find_downloaded_dataset(datasets_path: str, project_id: str, version_number: int) -> str:
        # logging.info(f"Directories in '{datasets_path}': {os.listdir(datasets_path)}")

        expected_name = f"{project_id}-{version_number}"
        pattern: Pattern[str] = re.compile(re.escape(pattern=expected_name), re.IGNORECASE)

        for folder in os.listdir(path=datasets_path):
            folder_path: str = os.path.join(datasets_path, folder)

            # logging.info(f"Comparing: '{folder}' vs '{expected_name}'")

            if os.path.isdir(s=folder_path) and pattern.search(string=folder):
                return folder_path 

        logging.error(f"Dataset not found for: '{project_id}-{version_number}' in '{datasets_path}'.")
        raise FileNotFoundError(f"Dataset not found for: '{project_id}-{version_number}' in '{datasets_path}'.")

In [24]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# trained_model = model.train(
#   data=CODE_DATASET_YAML_FILE,
#   epochs=300,
#   device='cuda',
#   batch=14,
#   patience=75,
#   optimizer="AdamW",
#   lr0=0.0005,  # Tasa de aprendizaje más baja para evitar falsos positivos
#   weight_decay=0.0005,  # Regularización para evitar sobreajuste
#   momentum=0.9,
#   warmup_epochs=5,
#   mixup=0.3,  # Más diversidad en el dataset
#   mosaic=0.7,  # Menos distorsión en imágenes
#   copy_paste=0.5  # Mejor detección de objetos difíciles
# )

class Model:
    def __init__(self, model: YOLO, data: str, epochs: int, device: str, batch: int, imgsz: int, optimizer: str) -> None:
        self.model = model
        self.data = data
        self.epochs = epochs
        self.device = device
        self.batch = batch
        self.imgsz = imgsz
        self.optimizer = optimizer
        self.lr0 = 0.001
        self.patience = None
        self.weight_decay = None
        self.momentum = None
        self.warmup_epochs = None
        self.mixup = None
        self.mosaic = None
        self.copy_paste = None
        
    def train(self) -> None:
        all_train_params: dict[str, str | int | float | None] = { 
            "data": self.data,
            "epochs": self.epochs,
            "device": self.device,
            "batch": self.batch,
            "patience": self.patience,
            "optimizer": self.optimizer,
            "lr0": self.lr0,
            "weight_decay": self.weight_decay,
            "momentum": self.momentum,
            "warmup_epochs": self.warmup_epochs,
            "mixup": self.mixup,
            "mosaic": self.mosaic,
            "copy_paste": self.copy_paste,
        }

        train_params: dict[str, str | int | float]  = {key: value for key, value in all_train_params.items() if value is not None}

        logging.info(msg=f"Training model with parameters: {train_params}")
        # self.model.train(**train_params)


class RoboflowDataset:
    def __init__(self, 
                datasets_path: str, 
                api_key: str, 
                the_workspace: str, 
                project_id: str, 
                version_number: int,
                model_format: str) -> None:

        self.datasets_path = datasets_path
        self.api_key = api_key
        self.the_workspace = the_workspace
        self.project_id = project_id
        self.version_number = version_number
        self.model_format = model_format
        self.dataset_path = None


    def download_dataset(self) -> None:
        try:
            os.chdir(path=self.datasets_path)
            rf: Roboflow = Roboflow(api_key=self.api_key)
            project: Project = rf.workspace(the_workspace=self.the_workspace).project(project_id=self.project_id)
            version: Version = project.version(version_number=self.version_number)
            dataset: Dataset = version.download(model_format=self.model_format)
        except:
            logging.error(msg=f"Something went wrong downloading the dataset")
            ValueError('Something went wrong downloading the dataset.')
    
    def find_dataset_path(self) -> str | None:
        try:
            dataset_path: str = DirectoryUtil.find_downloaded_dataset(datasets_path=self.datasets_path,
                                                                    project_id=self.project_id,
                                                                    version_number=self.version_number)
            logging.info(msg=f"Dataset is: {dataset_path}")
            self.dataset_path = dataset_path
            return dataset_path
        except:
            logging.error(msg=f"Something went wrong finding the dataset")
            ValueError('Something went wrong finding the dataset.')
    
    def prepare_dataset_datayml(self) -> None:
        if self.dataset_path is None:
            logging.error(msg=f"Dataset path is not set. Run find_dataset_path() first.")
            raise ValueError("Dataset path is not set. Run find_dataset_path() first.")

        dataset_datayml_path: str = os.path.join(self.dataset_path, 'data.yaml')
        # logging.info(msg=f"dataset_datayml_path {dataset_datayml_path}")

        with open(file=dataset_datayml_path, mode='r') as file:
            datayml_file: list[str] = file.readlines()

        # logging.info(msg=f"datayml_file {datayml_file} ")

        pattern: Pattern[str] = re.compile(r"https?://[^\s]+") 

        if(any(pattern.search(line) for line in datayml_file)):

            logging.info(msg=f"Dataset is not prepared. Preparing it...")
            datayml_file = datayml_file[:-4]
            logging.info(msg=f"Removed unneccesary content.")

            datayml_file = [line.replace("../train/images", f"{self.dataset_path}/train/images") for line in datayml_file]
            datayml_file = [line.replace("../valid/images", f"{self.dataset_path}/valid/images") for line in datayml_file]
            datayml_file = [line.replace("../test/images", f"{self.dataset_path}/test/images") for line in datayml_file]

            logging.info(msg=f"Added correct train, valid and test images path.")

            with open(file=dataset_datayml_path,mode= 'w') as f:
                f.writelines(datayml_file)

            logging.info(msg=f"Saved prepared file.")
        
        logging.info(msg=f"Dataset is ready to use for training.")

class Yolo:
    def __init__(self,
                models_dir: str,
                yolo_model: str = "yolov12l.pt",
                min_code_prediction_threshold: int = 3,
                min_diagram_prediction_threshold: int = 3) -> None:

        self.models_dir = models_dir
        self.yolo_model = yolo_model

        yolo_path: str = os.path.join(models_dir, "yolov12")
        if os.path.isdir(s=yolo_path):
            logging.info(msg="Yolo was found. Initializing.")
        else:
            logging.info(msg="Yolo was not found, dowloading it...")
            os.chdir(path=self.models_dir)
            repo_url: str = f"https://github.com/sunsmarterjie/yolov12"

            try:
                logging.info(msg="Downloading...")
                subprocess.run(["git", "clone", repo_url, yolo_path], check=True)
                logging.info(msg="Downloaded completed...")
            except subprocess.CalledProcessError:
                logging.error(msg="Cloud not download yolov12 from git. Check if git is installed.")
                RuntimeError("Cloud not download yolov12 from git. Check if git is installed.")
            
            commands: list[list[str]] = [
                ["pip", "install", "roboflow", "supervision", "flash-attn", "--upgrade", "-q"],
                ["pip", "install", "-r", "requirements.txt"],
                ["pip", "install", "-e", "."],
                ["pip", "install", "--upgrade", "flash-attn"],
                ["wget", f"https://github.com/sunsmarterjie/yolov12/releases/download/v1.0/{yolo_model}"]
            ]

            os.chdir(path=yolo_path)
            for command in commands:
                try:
                    logging.info(msg=f"Executing {' '.join(command)}")
                    subprocess.run(command, check=True)
                    logging.info(msg="Command executed successfully")
                except subprocess.CalledProcessError as e:
                    logging.info(msg=f"Error while executing {command}. Details: {e}")

        device: Literal['cuda', 'cpu'] = "cuda" if is_available() else "cpu"
        if device == 'cuda':
            logging.info(msg="GPU detected. Using it.")
        elif device == 'cpu':
            logging.info(msg="GPU not detected. Swichting to CPU.")
        
        self.code_model = None
        self.diagram_model = None
        self.min_code_prediction_threshold = min_code_prediction_threshold
        self.min_diagram_prediction_threshold = min_diagram_prediction_threshold
        self.valid_subjects = {"code", "diagram"}
        self.valid_images_extensions = {"jpg", "jpeg", "png", "webp"}
    
    def clear_cache(self) -> None:
        empty_cache()
        logging.info(msg="GPU cache has been cleared.")

    def is_subject_valid(self, subject: str) -> bool:
        """
        Checks if the given subject extension is valid.

        Args:
            subject (str): The subject (e.g., ".code").

        Returns:
            bool: True if the subject is valid, False otherwise.
        """
        return subject.lower() in self.valid_subjects

    def is_image_extension_valid(self, image_extension: str) -> bool:
        """
        Checks if the given image extension is valid.

        Args:
            image_extension (str): The image extension (e.g., ".png").

        Returns:
            bool: True if the extension is valid, False otherwise.
        """
        return image_extension.lower() in self.valid_images_extensions
    
    def format_detections_to_predictions(self, detections) -> dict[str, list[Any]]:
        formatted_predictions: list[Any] = []

        for i, (xyxy, conf, cls) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
            x_min, y_min, x_max, y_max = xyxy
            
            width = x_max - x_min
            height = y_max - y_min
            x_center = x_min + width / 2
            y_center = y_min + height / 2

            detection_id: str = str(uuid.uuid4())

            formatted_predictions.append({
                "x": round(number=float(x_center),ndigits=2),
                "y": round(number=float(y_center), ndigits=2),
                "width": round(number=float(width), ndigits=2),
                "height": round(number=float(height), ndigits=2),
                "confidence": round(number=float(conf),ndigits=3),
                "class": "code_snippet",
                "class_id": int(cls),
                "detection_id": detection_id
            })

        return {"predictions": formatted_predictions}

    def filter_images_with_code(self, images_path: Path) -> list[Path]:
        if self.code_model is None:
            raise ValueError("Code Model is not set. Run train_model(subject='code') or set_code_model(path='path/to/model') first.")

        valid_code_images_path: list[Path] = []

        all_images_paths: list[Path] = [file for file in images_path.iterdir() if self.is_image_extension_valid(image_extension=file.suffix)]

        for image_path in all_images_paths:
            image = cv2.imread(str(image_path))
            if image is None:
                logging.error(msg=f"Image: '{image_path.name}' could not be loaded. Skipping to the next image.")
                continue
                
            raw_code_detection_results = self.code_model(source=image, verbose=False)[0]
            code_detection_resutls: Detections = Detections.from_ultralytics(ultralytics_results=raw_code_detection_results).with_nms()
            code_predictions: dict[str, list[Any]] = self.format_detections_to_predictions(detections=code_detection_resutls)

            if len(code_predictions["predictions"]) > self.min_code_prediction_threshold:
                valid_code_images_path.append(image_path)

        return valid_code_images_path

    def filter_iamges_with_diagrams(self, images_path: Path) -> list[Path]:
        if self.diagram_model is None:
            raise ValueError("Diagram Model is not set. Run train_model(subject='code') or set_diagram_model(path='path/to/model') first.")

        valid_diagram_images_path: list[Path] = []

        all_images_paths: list[Path] = [file for file in images_path.iterdir() if self.is_image_extension_valid(image_extension=file.suffix)]

        for image_path in all_images_paths:
            image = cv2.imread(str(image_path))
            if image is None:
                logging.error(f"Image: '{image_path.name}' could not be loaded. Skipping to the next image.")
                continue
                
            raw_diagram_detection_results = self.diagram_model(source=image, verbose=False)[0]
            diagram_detection_resutls: Detections  = Detections.from_ultralytics(ultralytics_results=raw_diagram_detection_results).with_nms()
            diagram_predictions: dict[str, list[Any]] = self.format_detections_to_predictions(detections=diagram_detection_resutls)

            if len(diagram_predictions["predictions"]) > self.min_diagram_prediction_threshold:
                valid_diagram_images_path.append(image_path)

        return valid_diagram_images_path    
    
    def filter_images_with(self, subject: str, images_path: Path) -> None:
        if not self.is_subject_valid(subject):
            logging.error(msg=f"Subject is not valid. Only 'code' and 'diagram' are valid.")
            raise ValueError("Subject is not valid. Only 'code' and 'diagram' are valid.")

        match subject:
            case "code":
                self.filter_images_with_code(images_path)
            case "diagram":
                self.filter_iamges_with_diagrams(images_path)

    def train_code_model_with_dataset(self, roboflowDataset: RoboflowDataset) -> None:
        roboflowDataset.download_dataset()
        roboflowDataset.find_dataset_path()
        if(roboflowDataset.dataset_path is None):
            logging.error(msg="Dataset path is not set. Run find_dataset_path() or check your params.")
            raise ValueError("Dataset path is not set. Run find_dataset_path() or check your params.")

        roboflowDataset.prepare_dataset_datayml()

        model: Model = Model(model=self.code_model, data=roboflowDataset.dataset_path, epochs=300, device='cuda', batch=14, imgsz=640, optimizer="AdamW")
        model.train()

    # def train_model_with_and_dataset(self, subject: str) -> None:
        

NameError: name 'YOLO' is not defined

In [ ]:
MODELS_DIR: str = '/teamspace/studios/this_studio/models'
DATASET_DIR: str = '/teamspace/studios/this_studio/datasets'
API_KEY: str = os.environ["ROBOFLOW_API_KEY"]

KeyError: 'ROBOFLOW_API_KEY'

In [ ]:
models_dir: str = DirectoryUtil.ensure_directory(path=MODELS_DIR)

yolov12: Yolo = Yolo(models_dir=models_dir)

NameError: name 'DirectoryUtil' is not defined

In [62]:
datasets_path = DirectoryUtil.ensure_directory(path=DATASET_DIR)
api_key: str = API_KEY
the_workspace: str = 'bot-interactivo-tesis'
project_id: str = 'code-snippet-detection'
version_number: int = 22
model_format: str = 'yolov12'

codeRoboflowDataset: RoboflowDataset = RoboflowDataset(datasets_path=datasets_path,
                                                        api_key=api_key,
                                                        the_workspace=the_workspace,
                                                        project_id=project_id,
                                                        version_number=version_number,
                                                        model_format=model_format)

yolov12.train_code_model_with_dataset(roboflowDataset=codeRoboflowDataset)

2025-03-20 17:26:11,786 - INFO - Directory already exists: /teamspace/studios/this_studio/datasets


loading Roboflow workspace...
loading Roboflow project...


2025-03-20 17:26:13,245 - INFO - Dataset is: /teamspace/studios/this_studio/datasets/Code-Snippet-Detection-22
2025-03-20 17:26:13,246 - INFO - Dataset is ready to use for training.
